# Firefox Launcher Xpra Startup Debugging

This notebook helps diagnose issues with Xpra startup failures in the Firefox launcher extension. We'll check system dependencies, test Xpra manually, and investigate the server startup process.

The error we're seeing is: "Failed to launch Firefox via server proxy - Xpra startup failed" with a 500 Internal Server Error.

In [1]:
import os
import sys
import subprocess
import socket
import shutil
from pathlib import Path
from shutil import which
import asyncio
import json
import traceback
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("🔍 Firefox Launcher Debugging Setup")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")
print(f"User home: {Path.home()}")
print(f"Environment: {'JupyterHub' if os.environ.get('JUPYTERHUB_SERVICE_PREFIX') else 'Standalone'}")
print("=" * 60)

🔍 Firefox Launcher Debugging Setup
Python version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Working directory: /home/bdx/allcode/github/vantagecompute/jup-fir-lau
User home: /home/bdx
Environment: Standalone


## 1. System Dependencies Check

First, let's verify all required dependencies are available:

In [2]:
def check_system_dependencies():
    """Check for all required system dependencies."""
    deps = {
        'xpra': which('xpra'),
        'firefox': which('firefox'),
        'Xvfb': which('Xvfb'),
        'xauth': which('xauth'),
        'dbus-launch': which('dbus-launch')
    }
    
    print("🔍 System Dependencies Check:")
    all_present = True
    
    for dep_name, dep_path in deps.items():
        status = "✅" if dep_path else "❌"
        print(f"   {status} {dep_name}: {dep_path or 'NOT FOUND'}")
        if not dep_path:
            all_present = False
    
    if all_present:
        print("✅ All dependencies are present!")
    else:
        print("❌ Missing dependencies detected!")
        
    return deps, all_present

# Check dependencies
deps, deps_ok = check_system_dependencies()
print()

🔍 System Dependencies Check:
   ✅ xpra: /usr/bin/xpra
   ✅ firefox: /usr/bin/firefox
   ✅ Xvfb: /usr/bin/Xvfb
   ✅ xauth: /usr/bin/xauth
   ✅ dbus-launch: /usr/bin/dbus-launch
✅ All dependencies are present!



In [3]:
def check_versions():
    """Check versions of key dependencies."""
    print("🔍 Dependency Versions:")
    
    # Check Xpra version
    if deps['xpra']:
        try:
            result = subprocess.run([deps['xpra'], '--version'], 
                                  capture_output=True, text=True, timeout=5)
            xpra_version = result.stdout.strip() if result.returncode == 0 else "Error getting version"
            print(f"   🔧 Xpra: {xpra_version}")
        except Exception as e:
            print(f"   ❌ Xpra version check failed: {e}")
    
    # Check Firefox version
    if deps['firefox']:
        try:
            result = subprocess.run([deps['firefox'], '--version'], 
                                  capture_output=True, text=True, timeout=5)
            firefox_version = result.stdout.strip() if result.returncode == 0 else "Error getting version"
            print(f"   🦊 Firefox: {firefox_version}")
        except Exception as e:
            print(f"   ❌ Firefox version check failed: {e}")
    
    # Check Python packages
    try:
        import psutil
        print(f"   🐍 psutil: {psutil.__version__}")
    except ImportError:
        print("   ❌ psutil: Not installed")
    
    try:
        import tornado
        print(f"   🌪️ tornado: {tornado.version}")
    except ImportError:
        print("   ❌ tornado: Not installed")

check_versions()
print()

🔍 Dependency Versions:
   🔧 Xpra: xpra v5.1.1-r0
   🦊 Firefox: Mozilla Firefox 140.0.4
   🐍 psutil: 7.0.0
   🌪️ tornado: 6.5.1



## 2. Manual Xpra Testing

Let's test Xpra manually to see if it can start properly with our configuration:

In [4]:
def find_free_port():
    """Find a free port for testing."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

def setup_test_session_dir(port):
    """Set up session directory for testing."""
    session_dir = Path.home() / ".firefox-launcher" / "sessions" / f"test-session-{port}"
    
    # Create subdirectories
    socket_dir = session_dir / "sockets"
    runtime_dir = session_dir / "runtime"
    profile_dir = session_dir / "profile"
    temp_dir = session_dir / "temp"
    
    # Create all directories
    for dir_path in [session_dir, socket_dir, runtime_dir, profile_dir, temp_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Set permissions
    runtime_dir.chmod(0o700)
    for dir_path in [socket_dir, profile_dir, temp_dir]:
        dir_path.chmod(0o755)
    
    print(f"📁 Test session directory created: {session_dir}")
    return session_dir, socket_dir, runtime_dir, profile_dir, temp_dir

# Find a test port and set up directories
test_port = find_free_port()
print(f"🔌 Test port: {test_port}")

session_dirs = setup_test_session_dir(test_port)
session_dir, socket_dir, runtime_dir, profile_dir, temp_dir = session_dirs

print(f"📂 Session directories:")
for name, path in zip(['Session', 'Socket', 'Runtime', 'Profile', 'Temp'], session_dirs):
    print(f"   {name}: {path}")
print()

🔌 Test port: 39507
📁 Test session directory created: /home/bdx/.firefox-launcher/sessions/test-session-39507
📂 Session directories:
   Session: /home/bdx/.firefox-launcher/sessions/test-session-39507
   Socket: /home/bdx/.firefox-launcher/sessions/test-session-39507/sockets
   Runtime: /home/bdx/.firefox-launcher/sessions/test-session-39507/runtime
   Profile: /home/bdx/.firefox-launcher/sessions/test-session-39507/profile
   Temp: /home/bdx/.firefox-launcher/sessions/test-session-39507/temp



In [5]:
def check_firefox_wrapper_script():
    """Check if firefox-xstartup script is available."""
    print("🔍 Checking firefox-xstartup script:")
    
    # Check in PATH first
    firefox_wrapper = which("firefox-xstartup")
    if firefox_wrapper:
        print(f"   ✅ Found in PATH: {firefox_wrapper}")
        return firefox_wrapper
    
    # Check development location
    current_dir = Path.cwd()
    dev_script_path = current_dir / "scripts" / "firefox-xstartup"
    
    print(f"   🔍 Checking development location: {dev_script_path}")
    
    if dev_script_path.exists():
        print(f"   ✅ Found in development location: {dev_script_path}")
        
        # Check if executable
        if os.access(dev_script_path, os.X_OK):
            print("   ✅ Script is executable")
        else:
            print("   ⚠️ Script is not executable, attempting to fix...")
            try:
                os.chmod(dev_script_path, 0o755)
                print("   ✅ Made script executable")
            except Exception as e:
                print(f"   ❌ Failed to make executable: {e}")
        
        return str(dev_script_path)
    else:
        print(f"   ❌ Script not found in development location")
        print(f"   📁 Directory contents:")
        scripts_dir = current_dir / "scripts"
        if scripts_dir.exists():
            for item in scripts_dir.iterdir():
                print(f"      - {item.name}")
        else:
            print(f"      Scripts directory does not exist: {scripts_dir}")
        return None

firefox_wrapper = check_firefox_wrapper_script()
print()

🔍 Checking firefox-xstartup script:
   ✅ Found in PATH: /home/bdx/allcode/github/vantagecompute/jup-fir-lau/.venv/bin/firefox-xstartup



In [6]:
def create_test_xpra_command(port, session_dir, firefox_wrapper=None):
    """Create a simplified Xpra command for testing."""
    if not deps['xpra']:
        print("❌ Xpra not available")
        return None
    
    if not deps['Xvfb']:
        print("❌ Xvfb not available") 
        return None
    
    # Use the detected firefox wrapper or fallback to firefox directly
    start_child = firefox_wrapper if firefox_wrapper else deps['firefox']
    if not start_child:
        print("❌ No Firefox executable available")
        return None
    
    runtime_dir = session_dir / "runtime"
    
    # Create a simplified test command (without audio disabling for now)
    xpra_cmd = [
        deps['xpra'],
        "start",
        f"--bind-tcp=0.0.0.0:{port}",
        "--bind=none",
        "--html=on",
        "--daemon=no",
        "--exit-with-children=yes",
        "--start-via-proxy=no",
        "--start=",
        f"--start-child={start_child}",
        f"--xvfb={deps['Xvfb']} +extension Composite -screen 0 1280x800x24+32 -nolisten tcp -noreset +extension GLX",
        "--mdns=no",
        "--notifications=no",
        "--clipboard=yes",
        "--sharing=no",
        "--desktop-scaling=auto",
        "--resize-display=yes",
        "--use-display=no",
        f"--session-name=Test-Firefox-Session-{port}",
        f"--env=SESSION_DIR={session_dir}",
        f"--env=XDG_RUNTIME_DIR={runtime_dir}",
        "--env=PATH=/usr/local/bin:/usr/bin:/bin",
        "--dbus-launch=",
        "--dbus-proxy=no",
        "--remote-logging=no",
    ]
    
    print(f"📋 Test Xpra Command:")
    print(f"   Executable: {xpra_cmd[0]}")
    print(f"   Port: {port}")
    print(f"   Child process: {start_child}")
    print(f"   Session dir: {session_dir}")
    print(f"   Command length: {len(xpra_cmd)} arguments")
    
    return xpra_cmd

# Create test command
test_xpra_cmd = create_test_xpra_command(test_port, session_dir, firefox_wrapper)

if test_xpra_cmd:
    print("✅ Test command created successfully")
    print(f"🔧 Full command: {' '.join(test_xpra_cmd[:5])}... ({len(test_xpra_cmd)} total args)")
else:
    print("❌ Failed to create test command")
print()

📋 Test Xpra Command:
   Executable: /usr/bin/xpra
   Port: 39507
   Child process: /home/bdx/allcode/github/vantagecompute/jup-fir-lau/.venv/bin/firefox-xstartup
   Session dir: /home/bdx/.firefox-launcher/sessions/test-session-39507
   Command length: 25 arguments
✅ Test command created successfully
🔧 Full command: /usr/bin/xpra start --bind-tcp=0.0.0.0:39507 --bind=none --html=on... (25 total args)



In [7]:
async def test_xpra_startup(xpra_cmd, timeout=10):
    """Test Xpra startup and capture output."""
    if not xpra_cmd:
        print("❌ No command to test")
        return False, None, None, None
    
    print("🚀 Starting Xpra test process...")
    
    try:
        # Start the process
        process = subprocess.Popen(
            xpra_cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            env={
                **os.environ,
                "PATH": os.environ.get("PATH", ""),
                "XPRA_CRASH_DEBUG": "1",
            },
            preexec_fn=os.setsid if hasattr(os, "setsid") else None,
        )
        
        print(f"   ✅ Process started with PID: {process.pid}")
        
        # Monitor startup for a few seconds
        startup_checks = [0.5, 1.0, 2.0, 3.0]
        
        for i, wait_time in enumerate(startup_checks):
            await asyncio.sleep(wait_time)
            
            poll_result = process.poll()
            if poll_result is None:
                print(f"   ✓ Check {i+1}/{len(startup_checks)}: Process still running")
                
                # Test port connectivity after 2 seconds
                if i >= 1:
                    try:
                        test_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                        test_sock.settimeout(1.0)
                        result = test_sock.connect_ex(("localhost", test_port))
                        test_sock.close()
                        
                        if result == 0:
                            print(f"   🌐 Port {test_port} is accepting connections!")
                            # Process is running and port is open - success!
                            return True, process, None, None
                        else:
                            print(f"   🔍 Port {test_port} not yet ready (code: {result})")
                    except Exception as e:
                        print(f"   🔍 Port check failed: {e}")
            else:
                print(f"   ❌ Process exited with code: {poll_result}")
                break
        
        # If we get here, either process exited or timeout reached
        final_poll = process.poll()
        
        if final_poll is None:
            print(f"   ⏰ Process still running after {sum(startup_checks)}s but port not responding")
            print("   🛑 Terminating test process...")
            process.terminate()
            try:
                stdout, stderr = process.communicate(timeout=5)
            except subprocess.TimeoutExpired:
                process.kill()
                stdout, stderr = process.communicate()
        else:
            print(f"   ❌ Process exited with code: {final_poll}")
            try:
                stdout, stderr = process.communicate(timeout=5)
            except subprocess.TimeoutExpired:
                stdout, stderr = b"", b""
        
        return False, None, stdout, stderr
        
    except Exception as e:
        print(f"   💥 Exception during test: {e}")
        return False, None, None, str(e).encode()

# Run the test if we have a command
if test_xpra_cmd:
    print("🧪 Testing Xpra startup...")
    success, process, stdout, stderr = await test_xpra_startup(test_xpra_cmd)
    
    if success:
        print("🎉 Xpra startup test PASSED!")
        print("   The basic configuration works. Issue might be elsewhere.")
        
        # Clean up the running process
        if process:
            print("🛑 Cleaning up test process...")
            try:
                process.terminate()
                process.wait(timeout=5)
                print("   ✅ Test process terminated cleanly")
            except:
                process.kill()
                print("   💀 Test process force killed")
    else:
        print("❌ Xpra startup test FAILED!")
        
        if stdout:
            stdout_str = stdout.decode('utf-8', errors='ignore').strip()
            if stdout_str:
                print(f"📤 STDOUT ({len(stdout_str)} chars):")
                for line in stdout_str.split('\n'):
                    if line.strip():
                        print(f"     {line}")
        
        if stderr:
            stderr_str = stderr.decode('utf-8', errors='ignore').strip()
            if stderr_str:
                print(f"📤 STDERR ({len(stderr_str)} chars):")
                for line in stderr_str.split('\n'):
                    if line.strip():
                        print(f"     {line}")
else:
    print("❌ Cannot test - no command available")

print()

🧪 Testing Xpra startup...
🚀 Starting Xpra test process...
   ✅ Process started with PID: 396379
   ✓ Check 1/4: Process still running
   ✓ Check 2/4: Process still running
   🔍 Port 39507 not yet ready (code: 111)
   ✓ Check 3/4: Process still running
   🌐 Port 39507 is accepting connections!
🎉 Xpra startup test PASSED!
   The basic configuration works. Issue might be elsewhere.
🛑 Cleaning up test process...
   ✅ Test process terminated cleanly



## 3. Test with Full Audio Disabling (Current Configuration)

Let's test with all the audio-related options that were recently added:

In [ ]:
def create_full_audio_disabled_command(port, session_dir, firefox_wrapper=None):
    """Create Xpra command with full audio disabling - matches current configuration."""
    if not deps['xpra'] or not deps['Xvfb']:
        return None
    
    start_child = firefox_wrapper if firefox_wrapper else deps['firefox']
    if not start_child:
        return None
    
    runtime_dir = session_dir / "runtime"
    
    # This matches the current configuration in firefox_handler.py
    xpra_cmd = [
        deps['xpra'],
        "start",
        f"--bind-tcp=0.0.0.0:{port}",
        "--bind=none",
        "--html=on",
        "--daemon=no",
        "--exit-with-children=yes",
        "--start-via-proxy=no",
        "--start=",
        f"--start-child={start_child}",
        f"--xvfb={deps['Xvfb']} +extension Composite -screen 0 1280x800x24+32 -nolisten tcp -noreset +extension GLX",
        "--mdns=no",
        "--pulseaudio=no",       # Current config
        "--audio=no",            # Recently added
        "--sound=no",            # Recently added  
        "--notifications=no",
        "--clipboard=yes",
        "--clipboard-direction=both",
        "--sharing=no",
        "--speaker=off",         # Current config
        "--microphone=off",      # Current config
        "--webcam=no",
        "--desktop-scaling=auto",
        "--resize-display=yes",
        "--cursors=yes",
        "--bell=no",
        "--system-tray=no",
        "--xsettings=yes",
        "--readonly=no",
        "--window-close=auto",
        "--dpi=96",
        "--compressors=none",
        "--quality=100",
        "--encoding=auto",
        "--min-quality=30",
        "--min-speed=30",
        "--auto-refresh-delay=0.15",
        "--fake-xinerama=auto",
        "--use-display=no",
        f"--session-name=Test-Firefox-Audio-Session-{port}",
        "--env=PATH=/usr/local/bin:/usr/bin:/bin",
        f"--env=SESSION_DIR={session_dir}",
        f"--env=XDG_RUNTIME_DIR={runtime_dir}",
        f"--env=XAUTHORITY={runtime_dir}/.Xauth",
        "--env=XPRA_CLIPBOARD_WANT_TARGETS=1",
        "--env=XPRA_CLIPBOARD_GREEDY=1",
        "--dbus-launch=",
        "--dbus-proxy=no",
        "--remote-logging=no",
        "--bandwidth-detection=no",
        "--pings=yes",
    ]
    
    print(f"📋 Full Audio Disabled Command (Current Config):")
    print(f"   Audio options: --pulseaudio=no --audio=no --sound=no --speaker=off --microphone=off")
    print(f"   Command length: {len(xpra_cmd)} arguments")
    
    return xpra_cmd

# Test with full audio disabling
audio_test_port = find_free_port()
audio_session_dirs = setup_test_session_dir(audio_test_port)
audio_session_dir = audio_session_dirs[0]

print(f"🔌 Audio test port: {audio_test_port}")
audio_disabled_cmd = create_full_audio_disabled_command(audio_test_port, audio_session_dir, firefox_wrapper)

if audio_disabled_cmd:
    print("🧪 Testing Xpra with full audio disabling...")
    success, process, stdout, stderr = await test_xpra_startup(audio_disabled_cmd)
    
    if success:
        print("🎉 Audio disabled configuration test PASSED!")
        print("   Configuration is working correctly.")
        
        # Clean up
        if process:
            try:
                process.terminate()
                process.wait(timeout=5)
                print("   ✅ Test process cleaned up")
            except:
                process.kill()
    else:
        print("❌ Audio disabled configuration test FAILED!")
        print("   This suggests the recent audio changes might be causing issues.")
        
        if stdout:
            stdout_str = stdout.decode('utf-8', errors='ignore').strip()
            if stdout_str:
                print(f"📤 STDOUT:")
                for line in stdout_str.split('\n')[-10:]:  # Last 10 lines
                    if line.strip():
                        print(f"     {line}")
        
        if stderr:
            stderr_str = stderr.decode('utf-8', errors='ignore').strip()
            if stderr_str:
                print(f"📤 STDERR:")
                for line in stderr_str.split('\n')[-10:]:  # Last 10 lines
                    if line.strip():
                        print(f"     {line}")

print()

📁 Test session directory created: /home/bdx/.firefox-launcher/sessions/test-session-38937
🔌 Audio test port: 38937
📋 Full Audio Disabled Command (Current Config):
   Audio options: --pulseaudio=no --audio=no --sound=no --speaker=off --microphone=off
   Command length: 51 arguments
🧪 Testing Xpra with full audio disabling...
🚀 Starting Xpra test process...
   ✅ Process started with PID: 396937
   ❌ Process exited with code: 1
   ❌ Process exited with code: 1
❌ Audio disabled configuration test FAILED!
   This suggests the recent audio changes might be causing issues.
📤 STDERR:
     xpra initialization error:
      no such option: --sound



: 

## 4. Environment and Permission Checks

Let's check if there are any environment or permission issues:

In [ ]:
def check_environment_and_permissions():
    """Check environment variables and permissions."""
    print("🔍 Environment and Permission Checks:")
    
    # Check key environment variables
    env_vars = ['PATH', 'HOME', 'USER', 'DISPLAY', 'XDG_RUNTIME_DIR', 'TMPDIR', 'TEMP']
    for var in env_vars:
        value = os.environ.get(var, 'NOT SET')
        print(f"   {var}: {value}")
    
    # Check JupyterHub environment
    hub_vars = ['JUPYTERHUB_SERVICE_PREFIX', 'CONFIGPROXY_API_URL', 'CONFIGPROXY_AUTH_TOKEN']
    print("\n🔍 JupyterHub Environment:")
    for var in hub_vars:
        value = os.environ.get(var, 'NOT SET')
        masked_value = value if var != 'CONFIGPROXY_AUTH_TOKEN' else ('***' if value != 'NOT SET' else 'NOT SET')
        print(f"   {var}: {masked_value}")
    
    # Check session directory permissions
    print(f"\n🔍 Session Directory Permissions:")
    test_session_base = Path.home() / ".firefox-launcher" / "sessions"
    print(f"   Base sessions dir: {test_session_base}")
    
    if test_session_base.exists():
        try:
            # Try to create a test file
            test_file = test_session_base / "permission_test.txt"
            test_file.write_text("test")
            test_file.unlink()
            print("   ✅ Write permissions: OK")
        except Exception as e:
            print(f"   ❌ Write permissions: FAILED - {e}")
    else:
        print("   📁 Sessions directory doesn't exist yet")
    
    # Check disk space
    print(f"\n🔍 Disk Space:")
    try:
        import shutil
        total, used, free = shutil.disk_usage(Path.home())
        print(f"   Total: {total // (1024**3)} GB")
        print(f"   Used: {used // (1024**3)} GB") 
        print(f"   Free: {free // (1024**3)} GB")
        
        if free < 1024**3:  # Less than 1GB
            print("   ⚠️ Low disk space warning!")
    except Exception as e:
        print(f"   ❌ Error checking disk space: {e}")
    
    # Check running processes
    print(f"\n🔍 Current Firefox/Xpra Processes:")
    try:
        import psutil
        firefox_processes = []
        xpra_processes = []
        
        for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
            try:
                proc_name = proc.info['name'].lower()
                if 'firefox' in proc_name:
                    firefox_processes.append(f"PID {proc.info['pid']}: {proc.info['name']}")
                elif 'xpra' in proc_name:
                    xpra_processes.append(f"PID {proc.info['pid']}: {proc.info['name']}")
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
        
        print(f"   Firefox processes: {len(firefox_processes)}")
        for proc in firefox_processes[:3]:  # Show first 3
            print(f"      {proc}")
        if len(firefox_processes) > 3:
            print(f"      ... and {len(firefox_processes) - 3} more")
            
        print(f"   Xpra processes: {len(xpra_processes)}")
        for proc in xpra_processes[:3]:  # Show first 3
            print(f"      {proc}")
        if len(xpra_processes) > 3:
            print(f"      ... and {len(xpra_processes) - 3} more")
            
    except ImportError:
        print("   ❌ psutil not available for process checking")

check_environment_and_permissions()
print()

## 5. Server-Side Diagnostics

Let's check what might be happening on the server side when the extension tries to start Xpra:

In [ ]:
import asyncio
import aiohttp
import json

async def test_firefox_launcher_endpoint():
    """Test the Firefox launcher API endpoint directly."""
    print("🔍 Testing Firefox Launcher Endpoint:")
    
    # Get the base URL from environment or use default
    base_url = os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/user/bdx/')
    if not base_url.endswith('/'):
        base_url += '/'
    
    # Construct the API endpoint URL
    api_url = f"http://localhost:8889{base_url}firefox-launcher/api/firefox"
    print(f"   API URL: {api_url}")
    
    try:
        timeout = aiohttp.ClientTimeout(total=30)
        async with aiohttp.ClientSession(timeout=timeout) as session:
            
            # First try a GET request to check status
            print("   🔍 Testing GET request (status check)...")
            async with session.get(api_url) as response:
                print(f"   GET Status: {response.status}")
                if response.status == 200:
                    text = await response.text()
                    try:
                        data = json.loads(text)
                        print(f"   GET Response: {data}")
                    except:
                        print(f"   GET Response (first 200 chars): {text[:200]}...")
                elif response.status == 503:
                    print("   ✅ GET 503 is expected when no sessions are running")
                else:
                    print(f"   ⚠️ Unexpected GET status: {response.status}")
                    text = await response.text()
                    print(f"   Error response: {text[:500]}...")
            
            # Now try a POST request to start Firefox (this is what's failing)
            print("   🔍 Testing POST request (start Firefox)...")
            async with session.post(api_url) as response:
                print(f"   POST Status: {response.status}")
                text = await response.text()
                
                if response.status == 200:
                    try:
                        data = json.loads(text)
                        print(f"   🎉 POST Success: {data}")
                        
                        # If successful, we should clean up
                        port = data.get('port')
                        process_id = data.get('process_id')
                        if port and process_id:
                            print(f"   🧹 Cleaning up test session (port {port}, PID {process_id})")
                            cleanup_url = f"http://localhost:8889{base_url}firefox-launcher/api/cleanup"
                            cleanup_data = {"process_id": process_id, "port": port}
                            try:
                                async with session.post(cleanup_url, json=cleanup_data) as cleanup_resp:
                                    print(f"   🧹 Cleanup status: {cleanup_resp.status}")
                            except:
                                print("   ⚠️ Cleanup request failed (process may still be running)")
                        
                    except json.JSONDecodeError:
                        print(f"   ⚠️ POST response not JSON: {text[:200]}...")
                elif response.status == 500:
                    print(f"   ❌ POST Failed with 500 (Internal Server Error)")
                    try:
                        data = json.loads(text)
                        print(f"   Error details: {data}")
                    except:
                        print(f"   Error response: {text[:500]}...")
                else:
                    print(f"   ❌ POST Failed with status {response.status}")
                    print(f"   Response: {text[:500]}...")
                    
    except aiohttp.ClientError as e:
        print(f"   ❌ HTTP Client Error: {e}")
    except asyncio.TimeoutError:
        print(f"   ❌ Request Timeout")
    except Exception as e:
        print(f"   ❌ Unexpected Error: {e}")

# Test the endpoint
await test_firefox_launcher_endpoint()
print()

## 6. Cleanup and Summary

Let's clean up any test directories and summarize our findings:

In [ ]:
def cleanup_test_directories():
    """Clean up test session directories."""
    print("🧹 Cleaning up test directories:")
    
    sessions_base = Path.home() / ".firefox-launcher" / "sessions"
    if sessions_base.exists():
        test_dirs = [d for d in sessions_base.iterdir() if d.name.startswith('test-session-')]
        
        if test_dirs:
            for test_dir in test_dirs:
                try:
                    import shutil
                    shutil.rmtree(test_dir)
                    print(f"   ✅ Removed: {test_dir}")
                except Exception as e:
                    print(f"   ❌ Failed to remove {test_dir}: {e}")
        else:
            print("   📁 No test directories found")
    else:
        print("   📁 Sessions directory doesn't exist")

def print_debugging_summary():
    """Print a summary of debugging findings."""
    print("=" * 60)
    print("🔍 DEBUGGING SUMMARY")
    print("=" * 60)
    
    print("✅ What we checked:")
    print("   1. System dependencies (Xpra, Firefox, Xvfb)")
    print("   2. Version compatibility")
    print("   3. Basic Xpra command functionality")
    print("   4. Full audio-disabled configuration")
    print("   5. Environment variables and permissions")
    print("   6. Direct API endpoint testing")
    
    print("\n🎯 Common causes of 'Xpra startup failed':")
    print("   1. Missing system dependencies")
    print("   2. Permission issues with session directories")
    print("   3. Invalid Xpra command arguments")
    print("   4. Port conflicts")
    print("   5. Missing or non-executable firefox-xstartup script")
    print("   6. Environment variable issues")
    print("   7. Recent configuration changes (audio options)")
    
    print("\n🔧 Recommended next steps:")
    print("   1. Check the actual server logs for detailed error messages")
    print("   2. If basic test passed but full config failed, try removing recent audio options")
    print("   3. Verify the firefox-xstartup script exists and is executable")
    print("   4. Test Xpra manually from command line with similar arguments")
    print("   5. Check for conflicting processes or port usage")
    
    print("\n📋 Quick fixes to try:")
    print("   1. Restart JupyterLab/JupyterHub to clear any state issues")
    print("   2. Remove recent audio options (--audio=no, --sound=no) temporarily")
    print("   3. Check if firefox-xstartup script has correct permissions")
    print("   4. Clear session directories manually")

# Clean up and summarize
cleanup_test_directories()
print()
print_debugging_summary()